In [2]:
# -- coding: utf-8 --
from Environment import Environment
from model import First_procedure
from model import Second_procedure
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import pandas as pd
import seaborn as sbn
import numpy as np
from matplotlib.colors import ListedColormap
import copy
import pickle
import time
from multiprocessing import Pool
import random


In [6]:
'''
There are two class of parameters:

First parameters class are the parameters of the first-process--generate the anchor point, denoted by the inital letter 'f' 
f_time_distribution : type=str, the time displacement distribution of the first procedure. The parameters can choose ['u','exp','power']
f_time_parameters : type=[], the list of the parameters of the corresponded time distribution. for 'exp','power', f_time_parameters=[num],num<0
f_space_distribution : anchor point preferential choice of the center. The parameters can choose ['u','exp','power']
f_space_parameters = : the parameters of the f_space_distribution

Second are the parameters of the second-process -- expand the anchor point area and population,denoted by the inital letter 'f'
s_time_distribution: type=str, the time displacement distribution of the first procedure. The parameters can choose ['u','exp','power']
s_time_parameters: type=[], the list of the parameters of the corresponded time distribution. for 'exp','power', f_time_parameters=[num],num<0
step_r: the step distance every step, step_r=0.5 default
'''

def main(main_path,r,alpha,stop_step=1000000):

    f_time_distribution = 'exp'
    f_time_parameters = [1]
    f_space_distribution = 'u'
    f_space_parameters = [1]


    s_time_distribution = 'exp'
    s_time_parameters = [alpha]
    step_r=.5

    r=r
    ####--init the model-- ####
    first_process=First_procedure(f_time_parameters=f_time_parameters,f_time_distribution=f_time_distribution,f_space_parameters=f_space_parameters,f_space_distribution= f_space_distribution)
    second_process=Second_procedure(s_time_distribution,s_time_parameters,step_r)
    Envir = Environment(shape='square', r=r)

    anchor_point_flag=1
    t_now=0
    next_second_T=0
    next_first_T=0

    first_anchorpoint=first_process.cal(emptry_grid_list=Envir.get_empty_gridlist())
    flag=anchor_point_flag
    Envir.add_point(first_anchorpoint,flag=flag)
    Envir.anchorPoint.append(first_anchorpoint)

    simulation_clock=0


    #record invasion:
    change_steps=[]
    last_max_id=-1
    max_step=stop_step
    t_all=[0]
    flag_dict={}
    while(True):
        simulation_clock+=1
        next_second_T=second_process.get_next_T(Envir.point_num)
        next_first_T=first_process.get_next_T(Envir.get_anchorpoint_number())
        if(next_second_T>=next_first_T):
            t_now=next_first_T
        else:
            t_now=next_second_T

        if  t_now==next_first_T:
            ## first procedure
            anchor_point_flag += 1
            anchorpoint=first_process.cal(emptry_grid_list=Envir.get_empty_gridlist())
            if(anchorpoint==None):
                break
            flag=anchor_point_flag
            Envir.anchorPoint.append(anchorpoint)
            Envir.add_point(anchorpoint,flag)
        if t_now==next_second_T:
            point,flag = second_process.cal(gridDict=Envir.gridDict,rand_point=Envir.get_random_point())
            if point:
                Envir.add_point(point,flag=flag)
                # is invasion
                if(flag not in flag_dict):
                    flag_dict[flag]=0
                else:
                    flag_dict[flag]+=1
                pop_distribuition=flag_dict
                sorted_pop_distribution=sorted(pop_distribuition.items(),key=lambda x:x[0],reverse=True)
              
                if(sorted_pop_distribution[0][0]!=last_max_id):
                    t_sum=t_all[-1]+t_now
                    t_all.append(t_sum)
                    change_steps.append(simulation_clock)
                    last_max_id=sorted_pop_distribution[0][0]
                
                
        if(simulation_clock%(int(stop_step/50))==0):
            print (simulation_clock)
        if(simulation_clock>max_step):
            break

    file_name=main_path+'/'+str(time.mktime(time.localtime())+random.randint(1,100000))+'.pkl'
    values=[t_all,change_steps]
    with open(file_name,'wb+') as file:
        pickle.dump(values,file=file)
        print ('OK')


In [9]:
if __name__=='__main__':
    main_path='/home/wangjianying/data/Spatial_Yule/change_rate_without_m'
    alphalist=[2,5,10,20,50]
    
    for alpha in alphalist:      
        r=100
        print (alpha)
        simulate_num=100
        pool_num=20
        Pool_tool=Pool(pool_num)
        stop_step=1000000
        
        for i in range(simulate_num):
#             main(main_path,r,alpha,stop_step,)
            value=Pool_tool.apply_async(main,args=(main_path,r,alpha,stop_step,))
            print('Waiting for all subprocesses done...')
        Pool_tool.close()
        Pool_tool.join() 
        print('All subprocesses done.')
        
        import os 
        parts=os.listdir(main_path)
        paths=[main_path+'//'+path for path in parts if 'pkl' in path and str(stop_step)+'.pkl' not in path]
        temp_list=[]
        for path in paths:
            with open(path,'rb+') as file:
                print (path)
                temp_tempvalue=pickle.load(file)
                os.remove(path)
                temp_list.append(temp_tempvalue)
                
        outpath=main_path+'/'+str(alpha)+'_'+str(stop_step)+r'.pkl'
        with open(outpath,'wb+') as file:
            pickle.dump(temp_list,file)
        print ('All')

2
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...

660000
660000
680000
660000
660000
660000
680000
660000
680000
680000
680000
660000
680000
680000
680000
680000
680000
680000
680000
680000
680000
680000
700000
680000
680000
680000
700000
680000
700000
700000
700000
700000
680000
700000
700000
700000
700000
700000
700000
700000
700000
700000
720000
700000
700000
720000
700000
700000
720000
720000
720000
720000
700000
720000
720000
720000
720000
720000
720000
720000
720000
720000
740000
720000
720000
740000
720000
720000
740000
740000
740000
740000
740000
720000
740000
740000
740000
740000
740000
740000
740000
740000
760000
740000
740000
740000
760000
740000
760000
760000
760000
760000
760000
740000
760000
760000
760000
760000
760000
760000
760000
760000
780000
760000
760000
760000
780000
760000
780000
780000
780000
780000
780000
760000
780000
780000
780000
780000
780000
780000
780000
780000
780000
800000
780000
800000
780000
780000
800000
800000
800000
800000
800000
800000
800000
780000
800000
800000
800000
800000
800000
800000
800000

840000
840000
840000
840000
840000
840000
840000
840000
860000
860000
840000
860000
860000
780000
860000
840000
860000
840000
840000
860000
860000
860000
860000
860000
860000
860000
860000
860000
880000
880000
860000
880000
880000
880000
860000
800000
860000
880000
860000
880000
880000
880000
880000
880000
880000
880000
880000
880000
900000
900000
900000
880000
900000
900000
880000
880000
900000
880000
820000
900000
900000
900000
900000
900000
900000
900000
900000
900000
920000
920000
920000
900000
920000
920000
900000
900000
920000
900000
920000
920000
920000
920000
840000
920000
920000
920000
920000
920000
940000
940000
940000
920000
940000
940000
920000
920000
940000
920000
940000
940000
940000
940000
860000
940000
940000
940000
960000
940000
940000
960000
960000
960000
940000
960000
940000
940000
960000
940000
960000
960000
960000
960000
960000
960000
880000
960000
980000
960000
960000
980000
980000
980000
960000
980000
960000
960000
980000
960000
980000
980000
980000
980000
980000

OK
20000
20000
40000
60000
1000000
OK
1000000
OK
80000
20000
1000000
OK
1000000
80000
OK
20000
20000
40000
980000
40000
20000
60000
80000
100000
1000000
20000
OK
20000
40000
980000
20000
880000
1000000
OK
100000
40000
20000
40000
60000
60000
40000
80000
100000
40000
120000
40000
60000
20000
40000
1000000
OK
40000
120000
20000
60000
60000
80000
80000
60000
100000
120000
60000
60000
40000
140000
80000
60000
40000
20000
60000
80000
140000
80000
100000
1000000
OK
100000
80000
120000
140000
80000
60000
80000
100000
160000
1000000
OK
80000
900000
60000
40000
80000
100000
100000
160000
120000
120000
20000
100000
100000
140000
160000
80000
100000
120000
180000
100000
80000
60000
20000
100000
120000
120000
180000
140000
40000
140000
120000
120000
180000
160000
100000
120000
140000
120000
200000
100000
80000
120000
40000
140000
140000
60000
200000
160000
160000
140000
140000
200000
120000
180000
140000
920000
160000
140000
120000
100000
220000
140000
60000
160000
80000
160000
220000
180000
18000

180000
160000
140000
220000
240000
160000
200000
180000
240000
200000
920000
260000
160000
200000
220000
220000
200000
180000
220000
180000
200000
160000
220000
240000
940000
180000
200000
220000
260000
220000
260000
180000
280000
240000
220000
220000
240000
200000
240000
180000
200000
220000
960000
240000
260000
200000
220000
240000
240000
280000
200000
280000
240000
260000
240000
260000
300000
220000
200000
220000
260000
240000
980000
220000
260000
280000
240000
260000
260000
220000
300000
260000
300000
280000
260000
280000
320000
220000
240000
240000
1000000
OK
280000
260000
240000
260000
280000
300000
240000
280000
280000
20000
320000
280000
300000
240000
280000
300000
260000
260000
320000
340000
280000
300000
260000
40000
280000
300000
260000
300000
320000
300000
340000
300000
260000
320000
280000
280000
320000
300000
340000
360000
60000
300000
320000
280000
300000
280000
320000
320000
340000
320000
80000
280000
300000
320000
340000
360000
300000
340000
320000
360000
320000
380000

...

Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
W

660000
660000
660000
680000
680000
680000
680000
680000
680000
680000
680000
660000
680000
660000
680000
680000
680000
680000
680000
680000
660000
680000
680000
700000
700000
700000
700000
700000
680000
700000
700000
700000
700000
700000
680000
700000
700000
700000
700000
700000
700000
700000
720000
680000
720000
720000
720000
700000
720000
720000
720000
720000
720000
720000
720000
720000
700000
720000
720000
720000
720000
720000
740000
740000
740000
700000
740000
720000
740000
740000
740000
740000
740000
740000
740000
740000
740000
740000
740000
720000
740000
740000
760000
760000
760000
720000
760000
740000
760000
760000
760000
760000
760000
760000
760000
760000
760000
760000
740000
760000
760000
780000
760000
780000
780000
740000
780000
780000
780000
760000
780000
780000
780000
780000
780000
780000
780000
780000
760000
780000
800000
780000
800000
800000
780000
760000
800000
800000
780000
800000
800000
800000
800000
800000
800000
800000
800000
800000
800000
780000
820000
800000
820000

840000
820000
840000
840000
860000
860000
840000
840000
820000
840000
840000
880000
860000
840000
860000
820000
860000
840000
860000
860000
840000
840000
860000
860000
880000
880000
840000
860000
860000
860000
860000
900000
860000
880000
880000
840000
880000
860000
880000
880000
860000
880000
860000
880000
900000
900000
880000
860000
880000
880000
880000
920000
900000
880000
900000
860000
900000
880000
900000
900000
880000
900000
920000
880000
900000
900000
900000
900000
880000
920000
940000
900000
920000
900000
920000
920000
880000
920000
900000
920000
920000
900000
940000
920000
920000
900000
960000
920000
920000
920000
940000
900000
940000
940000
920000
940000
900000
940000
920000
940000
940000
960000
940000
940000
920000
980000
940000
920000
920000
940000
960000
940000
940000
960000
920000
960000
960000
960000
960000
940000
960000
940000
980000
960000
960000
1000000
OK
960000
960000
940000
980000
960000
960000
980000
940000
940000
980000
980000
980000
960000
980000
980000
960000
10

1000000
OK
20000
940000
20000
1000000
20000
OK
1000000
OK
1000000
OK
20000
20000
1000000
OK
980000
20000
20000
40000
40000
40000
40000
40000
20000
1000000
OK
980000
960000
40000
40000
20000
20000
40000
20000
40000
1000000
OK
20000
40000
40000
60000
60000
60000
60000
60000
40000
60000
1000000
OK
20000
40000
60000
980000
40000
60000
40000
60000
40000
20000
60000
80000
80000
80000
60000
80000
80000
60000
80000
40000
60000
80000
20000
80000
1000000
OK
60000
60000
80000
60000
40000
80000
100000
100000
100000
80000
100000
100000
80000
100000
60000
100000
80000
40000
100000
80000
80000
100000
80000
60000
20000
100000
120000
120000
100000
120000
120000
120000
100000
120000
80000
100000
120000
60000
120000
100000
120000
100000
100000
80000
120000
40000
140000
140000
120000
140000
140000
140000
120000
140000
120000
100000
140000
140000
140000
120000
120000
120000
80000
100000
140000
60000
160000
160000
160000
160000
140000
160000
140000
140000
160000
120000
160000
160000
160000
140000
140000
140

180000
160000
160000
220000
200000
60000
180000
140000
180000
220000
200000
180000
240000
200000
220000
200000
240000
200000
180000
160000
200000
180000
180000
220000
240000
80000
240000
200000
160000
220000
260000
200000
200000
260000
240000
220000
220000
220000
200000
180000
200000
220000
200000
240000
260000
100000
260000
220000
180000
280000
220000
240000
220000
280000
240000
260000
220000
240000
240000
200000
220000
240000
280000
220000
260000
280000
240000
120000
200000
300000
260000
240000
240000
300000
260000
240000
280000
260000
260000
220000
240000
260000
300000
240000
280000
300000
140000
280000
260000
320000
220000
280000
260000
320000
260000
260000
300000
280000
240000
260000
280000
320000
300000
260000
280000
320000
160000
300000
240000
280000
280000
340000
340000
280000
300000
280000
300000
260000
320000
280000
320000
280000
300000
340000
300000
340000
260000
180000
300000
300000
320000
360000
300000
320000
360000
320000
280000
300000
300000
340000
340000
300000
320000
3